In [9]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.io import read_image

class XRayImageDataset(Dataset):
    
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform # do we need this?
        self.target_transform = target_transform
        self.compute_mean_std()

    def compute_mean_std(self):
        #dataset = datasets.ImageFolder(root=self.img_dir)
        #images = torch.cat([img for img, _ in dataset], dim=0)
        means = []
        stds = []
        for filename in tqdm(os.listdir(self.img_dir)):
            img = read_image(os.path.join(self.img_dir, filename)) / 255
            mean, std = torch.std_mean(img)
            means.append(mean)
            stds.append(std)
            #print(img)
        #images = torch.cat([(read_image(os.path.join(self.img_dir, filename)) / 255) for filename in os.listdir(self.img_dir)])
        #self.mean, self.std = torch.std_mean(images, dim=0, keepdim=False)
        self.mean = np.mean(means)
        self.std = np.mean(stds)
        

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path) # PyTorch function, no need to change
        label = self.img_labels.iloc[idx, 2] # class_id column
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [ ]:
# print the head of the label csv


In [ ]:
from torch.utils.data import DataLoader

ROOT = "/kaggle/input/amia-public-challenge-2024"

test_img_path = ROOT + "/test/test"
train_img_path = ROOT + "/train/train"

test_annot_path = ROOT + "/test.csv"
train_annot_path = ROOT + "/train.csv"

print(test_img_path)

train_data = XRayImageDataset(train_annot_path, train_img_path)
print(f'Training images: mean {train_data.mean}, std {train_data.std}')
test_data = XRayImageDataset(test_annot_path, test_img_path)
print(f'Testing images: mean {test_data.mean}, std {test_data.std}')
#train_dataloader = DataLoader(train_data, batch_size=64, shuffle=True)
#test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

/kaggle/input/amia-public-challenge-2024/test/test


 50%|█████     | 4293/8573 [02:15<03:39, 19.47it/s]